In [31]:
import gym
from gym import spaces, error, utils
from gym.utils import seeding
import numpy as np
import copy
import random

In [32]:
N_TUBES = 3
N_COLORS = 2
H_TUBES = 3

DEFAULT_BOARD = [
    [1,1,2],
    [2,1,2],
    [0,0,0]
]

In [33]:
class BallSortEnv(gym.Env):
    def __init__(self, alpha=0.02):
        self.action_space = {0: (0,1),1:(0,2),2:(1,0),3:(1,2),4:(2,0),5:(2,1)}
        self.observation_space = {
            0 : 
[[1, 1, 2], [2, 1, 2], [0, 0, 0]] ,

1 : 
[[1, 1, 0], [2, 1, 2], [2, 0, 0]] ,

2 : 
[[1, 1, 2], [2, 1, 0], [2, 0, 0]] ,

3 : 
[[1, 1, 0], [2, 1, 0], [2, 2, 0]] ,

4 : 
[[1, 0, 0], [2, 1, 1], [2, 2, 0]] ,

5 : 
[[1, 1, 1], [2, 0, 0], [2, 2, 0]] ,

6 : 
[[1, 1, 1], [0, 0, 0], [2, 2, 2]] ,

7 : 
[[1, 1, 1], [2, 2, 0], [2, 0, 0]] ,

8 : 
[[1, 1, 0], [1, 0, 0], [2, 2, 2]] ,

9 : 
[[1, 1, 1], [2, 2, 2], [0, 0, 0]] ,

10 : 
[[1, 0, 0], [1, 1, 0], [2, 2, 2]] ,

11 : 
[[1, 1, 0], [2, 2, 2], [1, 0, 0]] ,

12 : 
[[0, 0, 0], [1, 1, 1], [2, 2, 2]] ,

13 : 
[[1, 0, 0], [2, 2, 2], [1, 1, 0]] ,

14 : 
[[2, 0, 0], [1, 1, 1], [2, 2, 0]] ,

15 : 
[[0, 0, 0], [2, 2, 2], [1, 1, 1]] ,

16 : 
[[2, 2, 0], [1, 1, 1], [2, 0, 0]] ,

17 : 
[[2, 0, 0], [2, 2, 0], [1, 1, 1]] ,

18 : 
[[2, 2, 2], [1, 1, 1], [0, 0, 0]] ,

19 : 
[[2, 2, 0], [2, 0, 0], [1, 1, 1]] ,

20 : 
[[2, 2, 2], [1, 1, 0], [1, 0, 0]] ,

21 : 
[[2, 2, 2], [0, 0, 0], [1, 1, 1]] ,

22 : 
[[2, 2, 2], [1, 0, 0], [1, 1, 0]] ,
        }
        self.alpha = alpha
        self.reset()

    def render(self, mode='human'):
        print(self.board)
        
        
    def step(self, action):
        if(self.checkGameOver()):
            done = True
            reward = 1.4
        else: 
            done = False
            reward = -0.1
        
        if(self.checkValidMove(action)):
            self.moveBall(action)
        
        info = {}

        return self.board, reward , done ,info
    
    def reset(self):
        self.board = copy.deepcopy(DEFAULT_BOARD)
        return self.board
    
    def close(self):
        pass
    
    def checkGameOver(self):
        numColsCompleted = 0
        for col in self.board:
            if(col.count(col[0]) == len(col) and col[0] != 0):
                numColsCompleted += 1
        if (numColsCompleted == N_COLORS):
            return True
        else: return False
    
    def checkValidMove(self,move):
        fromCol,toCol = move
        fromIndex = -1
        toIndex = -1
        for number in range(len(self.board[fromCol])-1,-1,-1):
            if(self.board[fromCol][number] != 0):
                fromIndex = number
                break
                    
        for number in range(0,len(self.board[toCol])):
            if(self.board[toCol][number] == 0):
                toIndex = number  
                break
    
        if(fromIndex == -1 or toIndex == -1):
            return False
        
        if(toIndex != 0):
            if(self.board[fromCol][fromIndex] != self.board[toCol][toIndex-1]):
                return False
        
        
        return True
    
    def moveBall(self,move):
        fromCol,toCol = move
        fromIndex = -1
        toIndex = -1
        for number in range(len(self.board[fromCol])-1,-1,-1):
            if(self.board[fromCol][number] != 0):
                fromIndex = number
                break
                    
        for number in range(0,len(self.board[toCol])):
            if(self.board[toCol][number] == 0):
                toIndex = number  
                break
                
        ball = self.board[fromCol][fromIndex]
        self.board[fromCol][fromIndex] = 0
        self.board[toCol][toIndex] = ball
        
def get_key(val,my_dict):
    for key, value in my_dict.items():
         if val == value:
             return key
    return -1
    

In [34]:
env = BallSortEnv()
action_space_size = len(env.action_space)
state_space_size = len(env.observation_space)


q_table = np.zeros((state_space_size, action_space_size))

print(q_table)


[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [35]:
num_episodes = 300
max_steps_per_episode = 20 # but it won't go higher than 1

learning_rate = 0.1
discount_rate = 0.99

exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01

exploration_decay_rate = 0.1 #if we decrease it, will learn slower

In [36]:
rewards_all_episodes = []

# Q-Learning algorithm
for episode in range(num_episodes):
    env.reset()
    
    state = 0
    
    done = False
    rewards_current_episode = 0
    
    for step in range(max_steps_per_episode):
        
        # Exploration -exploitation trade-off
        exploration_rate_threshold = random.uniform(0,1)
        if exploration_rate_threshold > exploration_rate:
            numAction = np.argmax(q_table[state,:])
            action = env.action_space[numAction]
        else:
            numAction = random.randint(0,5)
            #while(not env.checkValidMove(env.action_space[numAction])):
                #numAction = random.randint(0,5)
            action = env.action_space[numAction]    
            
            
            
        new_state, reward, done, info = env.step(action)
        new_index = get_key(new_state,env.observation_space)
        
        q_table[state, numAction] = (1 - learning_rate) * q_table[state, numAction] + \
            learning_rate * (reward + discount_rate * np.max(q_table[new_index,:]))
        
        state = new_index
        rewards_current_episode += reward
        
        if done == True: 
            break
    exploration_rate = min_exploration_rate + \
        (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate * episode)
    
    rewards_all_episodes.append(rewards_current_episode)
    
# Calculate and print the average reward per 10 episodes
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes), num_episodes / 100)
count = 100
print("********** Average  reward per thousand episodes **********\n")

for r in rewards_per_thousand_episodes:
    print(count, ": ", str(sum(r / 100)))
    count += 100
    
# Print updated Q-table
print("\n\n********** Q-table **********\n")
for table in range(0,len(q_table)):
    print(table," : ",q_table[table])

        

********** Average  reward per thousand episodes **********

100 :  0.6530000000000004
200 :  0.9980000000000007
300 :  0.9940000000000007


********** Q-table **********

0  :  [ 0.78354572 28.71820189 -0.12548956 -0.12111174 -0.13294534  2.02453402]
1  :  [ 2.65509201  1.54506009  2.03078141 30.17355252 -0.05302669 -0.09780271]
2  :  [-0.08491871  0.23107347 -0.08678801 -0.08504885 -0.08275207 -0.10567942]
3  :  [-0.0623164   4.85632375 31.64368165 -0.0499001  -0.04792594 -0.05367848]
4  :  [-0.04982293 -0.07753107 -0.05540748 -0.05698879 -0.05960069 -0.05960069]
5  :  [ 3.57998687e-02 -2.89848471e-02 -6.01766957e-02  3.31286787e+01
  2.48686352e+00 -3.43900000e-02]
6  :  [ 0.3794      0.37143568  1.29341598  2.15484858 34.62867849  0.        ]
7  :  [-0.01       -0.01       -0.01       -0.01893999 -0.019      -0.01      ]
8  :  [0. 0. 0. 0. 0. 0.]
9  :  [0.14 0.   0.   0.   0.   0.  ]
10  :  [0. 0. 0. 0. 0. 0.]
11  :  [0. 0. 0. 0. 0. 0.]
12  :  [0. 0. 0. 0. 0. 0.]
13  :  [0. 0. 0. 0

# 